# Исследование объявлений о продаже квартир

Данные сервиса Яндекс.Недвижимость — архив объявлений о продаже квартир в Санкт-Петербурге и соседних населённых пунктов за несколько лет. Нужно определить рыночную стоимость объектов недвижимости. Задача — установить параметры. Это позволит построить автоматизированную систему: отследить аномалии и мошенническую деятельность. 

По каждой квартире на продажу доступны два вида данных:
1. Вписаны пользователем.
2. Получены автоматически на основе картографических данных. Например, расстояние до центра, аэропорта, ближайшего парка и водоёма. 

## 1. Описание данных


* _airports_nearest_ — расстояние до ближайшего аэропорта в метрах (м)
* _balcony_ — число балконов
* _ceiling_height_ — высота потолков (м)
* _cityCenters_nearest_ — расстояние до центра города (м)
* _days_exposition_ — сколько дней было размещено объявление (от публикации до снятия)
* _first_day_exposition_ — дата публикации
* _floor_ — этаж
* _floors_total_ — всего этажей в доме
* _is_apartment_ — апартаменты (булев тип)
* _kitchen_area_ — площадь кухни в квадратных метрах (м²)
* _last_price_ — цена на момент снятия с публикации
* _living_area_ — жилая площадь в квадратных метрах(м²)
* _locality_name_ — название населённого пункта
* _open_plan_ — свободная планировка (булев тип)
* _parks_around3000_ — число парков в радиусе 3 км
* _parks_nearest_ — расстояние до ближайшего парка (м)
* _ponds_around3000_ — число водоёмов в радиусе 3 км
* _ponds_nearest_ — расстояние до ближайшего водоёма (м)
* _rooms_ — число комнат
* _studio_ — квартира-студия (булев тип)
* _total_area_ — площадь квартиры в квадратных метрах (м²)
* _total_images_ — число фотографий квартиры в объявлении

## 2. Импорт модулей

In [ ]:
import re
import pandas as pd
import seaborn as sns; sns.set()
import matplotlib.pyplot as plt

## 3. Функции

In [ ]:
def check_df_nan(data):
    """
    Получение количества пропусков в процентном соотношении (и типы данных 
    для каждого столбца с пропусками)
    data - таблица с данными
    """
    
    df_length = data.shape[0]
    null_stat = {}

    for col in data.columns:
        col_nan = data[data[col].isnull()].shape[0]
        pct = col_nan / df_length * 100

        if pct > 0:
            null_stat[col] = [round(pct, 1), data[col].dtype]
    
    info = (pd.DataFrame
     .from_dict(null_stat, orient='index', columns=['percentage', 'data_type'])
     .sort_values(by='percentage', ascending=False)
    )
    
    return info

In [ ]:
# добавим категории типа населенного пункта
def get_locality_category(name):
    options = ['деревня', 'снт', 'коттеджный посёлок', 'посёлок', 'село']
    
    for option in options:
        temp = r'^{}'.format(option)
        
        if re.findall(temp, name):
            return option
    
    if name == 'санкт-петербург':
        return 'мегаполис'
    
    return 'город'

In [ ]:
def check_center_nearest_nan(data, group, full=False):
    for gr, dt in df.groupby(group):
        nan_count = len(dt[dt['center_nearest'].isna()])
        total_count = len(dt)
        info = '{}\t{}/{}'.format(gr, nan_count, total_count)
        
        if full and nan_count == total_count:
            print(info)
            continue
            
        if nan_count:
            print(info)

In [ ]:
# функция определения категории этажа
def set_floor_category(floor, total):
    if floor == 1:
        return 'первый'
    
    if floor == total:
        return 'последний'
    else:
        return 'другой'

In [ ]:
# функция определения категории этажа в числовом обозначении
def set_floor_category_num(floor, total):
    if floor == 1:
        return 0
    
    if floor == total:
        return 1
    else:
        return 2

In [ ]:
# функция получения:
#   - описания указанных данных
#   - ущика с усами
#   - гистограммы указанных значений 

def get_data_info(data, column, dsc=True, boxplot=False, hist=False):
    cnt = 0
    print("Информация о данных столбца '{}'".format(column))
    
    short_data = data[[column]]
    
    if dsc:
        cnt += 1
        print("\n{}. Описание".format(cnt))
        print(short_data.describe())
        
    if boxplot:
        cnt += 1
        print("\n{}. Диаграмма размаха".format(cnt))
        short_data.plot.box(grid=True)
        plt.show()
        
    if hist:
        cnt += 1
        bins = len(short_data[column].unique())
        
        ylim = short_data.max() + short_data.std()
        Q1 = data[column].quantile(0.25)
        Q3 = data[column].quantile(0.75)
        IQR = Q3 - Q1
    
        p = Q1 - 1.5 * IQR
        llim = 0 if p < 0 else p
        rlim = Q3 + 1.5 * IQR
        
        print("\n{}. Гистограмма".format(cnt))
        short_data.plot.hist(bins=bins,
                             xlim=(llim,rlim),
                             grid=True,
                             legend=True,
                             title="Распределение '{}'".format(column)
                            )
        plt.show()

In [ ]:
# функция для построения графиков для списка параметров keys
def get_price_depend(data, keys, ylabel='last_price', title=None):
    if not title:
        title = "Зависимость цены {} от параметра '{}'"
        
    for key in keys:
        print(title.format(ylabel, key))
        data.plot.scatter(x=key, y=ylabel, c='c')
        plt.show()

In [ ]:
# функция для визуализации матрицы корреляции
def corr_visual(corr_df):
    return sns.heatmap(corr_df, linewidths=.5, cmap="coolwarm", annot=True)

## 4. Загрузка данных

In [ ]:
df = pd.read_csv('/datasets/real_estate_data.csv', sep='\t')
df.info()

In [ ]:
df.head()

In [ ]:
df['locality_name'].value_counts().sort_index()[160:170]

В данных выявлено:
* количество объектов - 23699
* имеются пропуски значений в 14 столбцах:
    - ceiling_height
    - floors_total
    - living_area
    - is_apartment
    - kitchen_area
    - balcony
    - locality_name
    - airports_nearest
    - cityCenters_nearest
    - parks_around3000
    - parks_nearest
    - ponds_around3000
    - ponds_nearest
    - days_exposition
* данные представлены для разных населенных пукнтов: города, посёлки, деревни, сёла
* значения представлены числами с плавающей точкой, строками,  целочисленными и булевыми значениями
* встречаются названия одинаковых населенных пунктов, написанных по-разному (_"поселок", "посёлок", "поселок городского типа"_ )

## 5. Предобработка данных

### 5.1 Анализ количества пропусков в данных

In [ ]:
check_df_nan(df)

### 5.2 Заполнение данных и изменение типов

**Признак _"last_price"_**

Пропусков нет. Переведем значения к целочисленному типу, как принято указывать цены на недвижимость.

In [ ]:
df['last_price'] = df['last_price'].astype('int')

# проверка
df['last_price'].dtypes

**Признак _"is_apartment"_**

Пропуски в признаке "is_apartment" скорее всего говорит о том, что  объект - не апартаменты.

In [ ]:
df['is_apartment'].fillna(False, inplace=True)

# проверка
df['is_apartment'].value_counts(dropna=False)

In [ ]:
# проверяем тип данных
df[['is_apartment']].dtypes

**Признак _"balcony"_**

Пропуски в признаке "balcony" скорее всего говорит о том, что балкона нет.

In [ ]:
df['balcony'].fillna(0, inplace=True)

# проверка
df['balcony'].value_counts(dropna=False)

In [ ]:
# переведем тип в целочисленный, так как для данного признака это более подходящий
df['balcony'] = df['balcony'].astype('int')

# проверяем тип данных
df[['balcony']].dtypes

**Признак _"locality_name"_**

Пропуски в признаке "locality_name" составляют всего 0.02%. В то же время, это независимый признак и непонятно, на что его заменить. Поэтому удалим записи с подобными пропусками.

In [ ]:
# получаем индексы с пропущенными названиями населенного пункта и удаляем по индексам записи
locality_missing_idx = df[df['locality_name'].isnull()].index
df = df.drop(locality_missing_idx, axis=0)

# проверка
df[df['locality_name'].isnull()].shape[0]

In [ ]:
# убираем неcогласованность данных в 'locality_name'
df['locality_name'] = df['locality_name'].str.lower()
df['locality_name'] = df['locality_name'].str.strip()

# заменяем вариации посёлка на "посёлок"
temp = r'поселок|посёлок городского типа|городской посёлок|посёлок станции'
df['locality_name'] = df['locality_name'].str.replace(temp, 'посёлок')
df['locality_name'] = df['locality_name'].str.replace('при железнодорожной станции ', '')

# заменяем вариации СНТ на "снт"
temp = r'садоводческое некоммерческое товарищество|садовое товарищество'
df['locality_name'] = df['locality_name'].str.replace(temp, 'снт')

# проверка
temp = r'деревня|^посёлок|снт|коттеджный посёлок|село'
df[~(df['locality_name'].str.contains(temp))]['locality_name'].sort_values().unique()

In [ ]:
df['locality_category'] = df['locality_name'].apply(get_locality_category)
df['locality_category'].value_counts()

**Признак _"floors_total"_**

Пропуски в признаке "floors_total" заполним на медианные значения для каждого города, так как группировка по _типу_ населенного пункта будет слишком крупной.

In [ ]:
locality_floors_stat = df.groupby('locality_name')['floors_total'].median()
locality_floors_stat.head(10)

In [ ]:
for col in locality_floors_stat.index:
    med = locality_floors_stat[col]
    mask = df['locality_name'] == col 
    df.loc[mask,'floors_total'] = df.loc[mask,'floors_total'].fillna(med)

# проверка
df[df['floors_total'].isna()].shape[0]

In [ ]:
# количество этажей должно быть целочисленным значением
df['floors_total'] = df['floors_total'].astype('int')
df['floors_total'].dtypes

**Признак _"rooms"_**

Если комнат 0, значит это или пропуск,  или открытая планировка.

In [ ]:
# общее количество пропусков
zero_rooms = df[df['rooms'] == 0]['rooms'].count()
zero_rooms

In [ ]:
# сколько из 0-комнатных - с открытой планировкой
zero_rooms_open = df[(df['rooms'] == 0) & (df['open_plan'] == True)]['open_plan'].value_counts()
zero_rooms_open

In [ ]:
# являются ли студией 0-комнатные квартиры без открытой планировки
zero_rooms_studio = df[(df['rooms'] == 0) & (df['open_plan'] == False)]['studio'].value_counts()
zero_rooms_studio

In [ ]:
zero_rooms_open + zero_rooms_studio 

Если 0-комнатные квартиры без открытой планировки, то это студия и заодно - неявный пропуск. Заменим 0 значения комнат для студий на 1.

In [ ]:
mask = ((df['rooms'] == 0) & (df['studio'] == True))
df.loc[mask, 'rooms'] = df.loc[mask, 'rooms'].replace(to_replace=0, value=1)

# проверка
print(df[(df['rooms'] == 0) & (df['studio'] == True)].shape[0])

**Признаки _"kitchen_area"_ и  _"living_area"_**

In [ ]:
# количество пропусков
print('kitchen_area:', df[df['kitchen_area'].isna()].shape[0])
print('living_area: ', df[df['living_area'].isna()].shape[0])

In [ ]:
# если открытая планировка или студия, то кухня будет 0
col_list = ['kitchen_area', 'living_area']
mask = ((df['open_plan'] == True) | (df['studio'] == True))
df.loc[mask, col_list] = df.loc[mask, col_list].fillna(0)

# остальное
print('Осталось пропусков')
print('kitchen_area - ', df[df['kitchen_area'].isna()].shape[0])
print('living_area  - ', df[df['living_area'].isna()].shape[0])

In [ ]:
# пропуски kitchen_area, сгруппированные по количеству комнат
df[df['kitchen_area'].isna()]['rooms'].value_counts().sort_index()

In [ ]:
# пропуски living_area, сгруппированные по количеству комнат
df[df['living_area'].isna()]['rooms'].value_counts().sort_index()

In [ ]:
# в отличие от living_area, kitchen_area имеет пропуски и для 8-комнатных квартир
# создадим общий список для последующего заполнения пропусков, возьмем его от kitchen_area

room_groups = df[df['kitchen_area'].isna()]['rooms'].value_counts().index

In [ ]:
# Найдем медиану площади кухни для квартир с разным количеством комнат
def get_area_stat(data, group, column):
#     stat = df[~(df[column].isna()) & (df[group] != 0)].groupby(group).agg({column:['median', 'mean']})
    stat = df[~(df[column].isna())].groupby(group).agg({column:['median', 'mean']})
    stat.columns = ['area_median', 'area_mean']
    return stat
    
kitchen_area_stat = get_area_stat(df, 'rooms', 'kitchen_area')
living_area_stat = get_area_stat(df, 'rooms', 'living_area')

kitchen_area_stat

In [ ]:
living_area_stat

In [ ]:
for group in room_groups:
    med_living = living_area_stat.loc[group, 'area_median']
    med_kitchen = kitchen_area_stat.loc[group, 'area_median']
    mask = (df['rooms'] == group)
    
    df.loc[mask, 'kitchen_area'] = df.loc[mask, 'kitchen_area'].fillna(med)
    df.loc[mask, 'living_area']  = df.loc[mask, 'living_area'].fillna(med)
    
print('Осталось пропусков')
print('kitchen_area - ', df[df['kitchen_area'].isna()].shape[0])
print('living_area  - ', df[df['living_area'].isna()].shape[0])

**Признак _"ceiling_height"_**

In [ ]:
# количество пропусков
df[df['ceiling_height'].isna()].shape[0]

In [ ]:
ceiling_height_stat = df[~(df['ceiling_height'].isna())]
ceiling_height_stat[['ceiling_height']].describe()

In [ ]:
ceiling_height_stat[['ceiling_height']].boxplot(figsize=(10,12))

In [ ]:
# посмотрим на данные с высотой потолка > 5

columns = ['ceiling_height', 'total_area', 'open_plan', 'studio', 'locality_category']
df.query('ceiling_height > 5')[columns].sort_values(by='ceiling_height')

Видно много одинаковых значений 24, 25, 26, 27, 32 что говорит о том, что скорее всего произошла ошибка на порядок, что можно исправить.

In [ ]:
mask = ((df['ceiling_height'] >= 24.0) & (df['ceiling_height'] <= 32.0))
df.loc[mask, 'ceiling_height'] = df.loc[mask, 'ceiling_height'] / 10

df.query('ceiling_height > 5')[columns].sort_values(by='ceiling_height')

Квартиры с площадью больше 100 кв.м. могут быть двухуровневыми, оставим их без изменений.

In [ ]:
df.query('ceiling_height > 5 and total_area < 100')[columns].sort_values(by='ceiling_height')

Значение потолка 100 - выброс, удалим эту запись (возможно, было перепутано значение при заполнении объявления).

In [ ]:
df.drop(index=df[df['ceiling_height'] == 100.0].index,inplace=True)

# проверка
df[df['ceiling_height'] == 100.0].shape[0]

In [ ]:
# обновленная табица
ceiling_height_stat = df[~(df['ceiling_height'].isna())]
ceiling_height_stat[['ceiling_height']].boxplot()

Из диаграмы видно, что есть выбросы как в меньшую, так и в большую стороны - квартиры с высотой 1 м или выше 5 метров выглядят неправдоподобно.

In [ ]:
ceiling_height_stat[['ceiling_height']].describe()

In [ ]:
# Посмотрим на медианные значения высоты потолка для каждой этажности
ceiling_pivot = pd.pivot_table(ceiling_height_stat, values='ceiling_height', index='floors_total',
                               aggfunc=['mean', 'median'])
ceiling_pivot.columns = ['ceiling_mean', 'ceiling_median']
ceiling_pivot

In [ ]:
# заполним медианой для каждой этажности, так как этот параметр, как правило обозначает тип застройки
for floor in ceiling_pivot.index:
    mask = (df['floors_total'] == floor)
    value = ceiling_pivot.loc[floor, 'ceiling_median']
    
    df.loc[mask, 'ceiling_height'] = df.loc[mask, 'ceiling_height'].fillna(value)

# проверка
print(df[df['ceiling_height'].isna()].shape[0])
df[df['floors_total'] == 33][['ceiling_height', 'floors_total']]

Остался один пропуск для 33-этажного дома в Санкт-Петербурге. Заменим его медианой по высоте потолка (одно значение, на анализ это не повлияет)

In [ ]:
m = (df['floors_total'] == 33)
med = df['ceiling_height'].median()
df.loc[m, 'ceiling_height'] = df.loc[m, 'ceiling_height'].fillna(med)

# проверка
df[df['ceiling_height'].isna()].shape[0]

<div class="alert alert-warning">
<h2> Комментарий наставника</h2>

Заполнение такого числа пропусков в высоте потолков одним значением может исказить структуру данного столбца. Слишком много пропусков у нас в данном параметре. Подумай, на основании какого параметра лучше будет обработать данный столбец. Строительство, в основном, типовое, поэтому одинаковые дома обладают схожими характеристиками. 

</div>

In [ ]:
df.info()

**Признак _"first_day_exposition"_**

In [ ]:
# в признаке нет пропусков, изменим тип object -> datetime
df['first_day_exposition'] = pd.to_datetime(df['first_day_exposition'],format='%Y-%m-%dT%H:%M:%S')

print(df['first_day_exposition'].dtypes)

**Признак _"days_exposition"_**

In [ ]:
# количество пропусков
df[df['days_exposition'].isna()].shape[0]

In [ ]:
keys= ['total_images', 'last_price', 'total_area',
       'locality_name','days_exposition', 'locality_category']

df[df['days_exposition'].isnull()][keys].sort_values('days_exposition')[:15]

1. Пропуски признака составляют 13.5% от общего числа записей.
2. Согласно правилам размещения объявления о продаже (https://yandex.ru/support/realty/paid.html), существует 2 вида объявлений: бесплатные и платные.
3. Для платных объявлений включается автопродление. Для бесплатных - срок аренды объявления для продажи в Санкт-Петербурге и Ленинградской области составляет 90 дней.

Посмотрим, сколько объявлений с таким сроком.

In [ ]:
len(df[df['days_exposition'] == 90][['days_exposition']].sort_values('days_exposition', ascending=False))

Нисколько.

Так как признак того, что квартира была продана, отсутствует, можно предположить, что для объектов, у которых закончился бесплатный период размещения, существует отдельный признак, обозначающий этот момент и не попавший в исследуемый датасет.

Если заполнить пропущенные значения максимальным значением срока аренды для Санкт-Петербуга и области (90 дней), будет аномально большое количество с таким сроком. Заполним эти данные 0.

In [ ]:
#  посмотрим на количество объявлений по годам

df['first_day_exposition'].dt.year.plot.hist(bins=6, legend=True)

In [ ]:
# ДОПОЛНЕНИЕ

# посмотрим на распределение пропусков в столбце 'days_exposition' во времени:
# относительно дняб месяца и года выставления квартиры на продажу.

days_exposition_null = df[df['days_exposition'].isnull()]
days_exposition_null['year'] = days_exposition_null['first_day_exposition'].dt.year
days_exposition_null['month'] = days_exposition_null['first_day_exposition'].dt.month
days_exposition_null['day'] = days_exposition_null['first_day_exposition'].dt.weekday + 1

for val in ('year', 'month', 'day'):
    bins = len(days_exposition_null[val].unique())
    lim = days_exposition_null[val].value_counts().max() + 200
    days_exposition_null[val].plot.hist(bins=bins,
                                        ylim=(0,lim),
                                        legend=True,
                                        title='Распределение пропусков по \'{}\''.format(val)
                                       )
    plt.show()
    

Из графиков видно, что с каждым годом количество пропусков увеличивается, что говорит связано скорее всего с увеличением количества объявлений. При распределении по месяцам закономерностей нет. По дням недели видно, что в первые 3 дня больше всего пропусков, меньше в 4 и 5 дни и минимальное количество - 6 и 7 дни (выходные). Заполним пропуски медианным значением 'days_exposition' для каждой группы дня недели.

In [ ]:
df['day'] = df['first_day_exposition'].dt.weekday + 1

df.groupby('day')[['days_exposition']].median()

In [ ]:
# # df['days_exposition'] = df['days_exposition'].fillna(90)
# df['days_exposition'] = df['days_exposition'].fillna(0)

# ----------------------------
# ДОПОЛНЕНИЕ


for day, data in df.groupby('day'):
    mask = (df['day'] == day)
    value = data['days_exposition'].median()
    
    df.loc[mask, 'days_exposition'] = df.loc[mask, 'days_exposition'].fillna(value)


# проверка
df[df['days_exposition'].isna()].shape[0]

In [ ]:
# # изменим тип данных float64 -> int64
# df['days_exposition'] = df['days_exposition'].astype('int')

# проверка
print(df['days_exposition'].dtypes)

<div class="alert alert-danger">
<h2> Комментарий наставника</h2>

Заполнять пропуски в данных отвлеченными значениями - это не очень правильное решение. Во-первых, ты можешь просто забыть их отфильтровать. Заполнять нужно в том случае, когда алгоритм, для которого готовятся данные, чувствителен к пропускам. Например, регрессии чувствительны, а корреляция  нет. Для них заполнение может даже навредить, если заполнять маркерными значениями и забыть о фильтрации.

Заполнять мы, строго говоря, имеем право только когда данные пропущены совершенно случайно (MCAR) - тогда заполнение не повлияет на характер связи между заполняемой переменной и остальными. Относятся ли пропуски в данном столбце к данному типу пропусков? Посмотри внимательнее на распределение пропусков в данном столбце во времени, т.е. относительно дня выставления квартиры на продажу. 


</div>

<font color=orange>Обоснование своих действий ты привела. Однако графики нам показывают, что пропуски не являются случайными. Данные квартиры еще находятся на этапе продажи. Если ты хочешь предсказать эти значения, то стоит использовать самые коррелярующие параметры, а не дни недели. Заполнение по дням недели не учитывает специфику продаваемой квартиры, а значит слабым образом влияет на срок продажи. 
    
---

**Признак _"airports_nearest"_**

В Ленинградской области только 2 аэропорта (пассажирских) - Пулково-1 и Пулково-2. Посмотрим на пропуски для областного центра и других городов.

In [ ]:
df.airports_nearest.isna()

In [ ]:
piter = df[(df.locality_name == "Санкт-Петербург") & (df.airports_nearest.isna())].shape[0]
other = df[(df.locality_name != "Санкт-Петербург") & (df.airports_nearest.isna())].shape[0]
max_distance = df['airports_nearest'].sort_values(ascending=False).max()

print('Пропуски airports_nearest для Санкт-Петербурга:', piter)
print('Пропуски airports_nearest для других н.с.:     ', other)
print('Максимальное расстояние до аэропорта в данных: ', max_distance)

Отсутсвие данных для признака говорит о том, что аэропорт далеко. Судя по максимальному значению, можно предположить, что согласно алгоритму сбора картографических данных, поиск ближайшего аэропорта осуществляется в радиусе 100 км. Заполним пропуски в признаке большим числом - 110000.

In [ ]:
fill_value = 110000
df.loc[df['airports_nearest'].isna(), 'airports_nearest'] = \
    df.loc[df['airports_nearest'].isna(), 'airports_nearest'].fillna(fill_value)

# проверка
df[df['airports_nearest'].isna()].shape[0]

**Признаки _"parks_around3000", "ponds_around3000"_**

In [ ]:
# посмотрим на объединение и пересечение пропусков в этих признаках
print(df[(df['parks_around3000'].isna()) | (df['ponds_around3000'].isna())].shape[0])
print(df[(df['parks_around3000'].isna()) & (df['ponds_around3000'].isna())].shape[0])

Вывод - пропуски для одних и тех же объектов.
Посмотрим на населенные пункты с отсутствующими парками и водоемами в радиусе 3 км.

In [ ]:
parks_ponds_null = df[(df['parks_around3000'].isna()) | (df['ponds_around3000'].isna())]

print("Пропуски для Санкт-Петербурга:",
      parks_ponds_null.query('locality_name == "Санкт-Петербург"').shape[0])

In [ ]:
print('Категории н.с. с пропусками')

parks_ponds_null['locality_category'].value_counts()

Если не указаны водоемы и парки в радиусе 3 км, то скорее всего этих объектов поблизости нет. Также объекты могут находиться в таких населенных пунктах, где таких объектов вообще нет: СНТ, сёла, коттеджные посёлки. Заполним пропуски 0.

In [ ]:
for obj in ['parks_around3000', 'ponds_around3000']:
    df[obj] = df[obj].fillna(0)
    
# проверка
print(len(df[df['parks_around3000'].isna()]), len(df[df['ponds_around3000'].isna()]))

**Признаки _"parks_nearest", "ponds_nearest"_**

In [ ]:
parks_nearest = df[['parks_nearest']].sort_values(by='parks_nearest', ascending=False)
ponds_nearest = df[['ponds_nearest']].sort_values(by='ponds_nearest', ascending=False)

print("Наиболее удаленные:")
print("Парки:   {} м".format(parks_nearest['parks_nearest'].max()))
print("Водоемы: {} м".format(ponds_nearest['ponds_nearest'].max()))

In [ ]:
parks_nearest.describe()

In [ ]:
ponds_nearest.describe()

Для парков минимальное значение - 1 метр от парка, максимальное - 3190, стандартное отклонение - почти равно медиане. Это допустимо, учитывая, что в данных разные типы населенных пунктов. Заполним пропуски в признаках большим значением 10000 м, которое будет означать, что ближайшие парки и водоемы далеко.

In [ ]:
df[['parks_nearest', 'ponds_nearest']] = df[['parks_nearest', 'ponds_nearest']].fillna(10000)
  
# проверка
print(df[df['parks_nearest'].isna()].shape[0])
print(df[df['ponds_nearest'].isna()].shape[0])

**Признак _"cityCenters_nearest"_**

In [ ]:
# переименуем столбец
df.rename(columns={"cityCenters_nearest": "center_nearest"}, inplace=True)

# проверка
print([col for col in df.columns if 'nearest' in col])

In [ ]:
df[df['locality_name'] == "санкт-петербург"]['center_nearest'].min()

In [ ]:
df[df['locality_name'] == "санкт-петербург"]['center_nearest'].max()

In [ ]:
# Посмотрим на категории нас. пункта с пропусками
df[df.center_nearest.isna()]['locality_category'].value_counts()

In [ ]:
# соотношение количества пропусков к числу записей для каждой категории н.п.
check_center_nearest_nan(df, 'locality_category')

Расстояние до центра польностью отсутствует для __категорий__ населенных пунктов:
* деревня
* коттеджный посёлок
* село
* снт

Для таких категорий заполним пустые значения 0, так как это малые населенные пункты.

In [ ]:
keys = ['деревня', 'коттеджный посёлок', 'село', 'снт']
mask = (df['locality_category'].isin(keys))

df.loc[mask, 'center_nearest'] = df.loc[mask, 'center_nearest'].fillna(0)

# проверка оставшихся пропусков
check_center_nearest_nan(df, 'locality_category')

Посмотрим, есть ли __населенные пункты__ , для которых полностью отсутствуют данные о расстоянии до центра.

In [ ]:
check_center_nearest_nan(df, 'locality_name', full=True)

Для многих населенных пунктов полностью отсутствуют значения. Заполним медианой по категориям населенных пунктов.

In [ ]:
for gr in df['locality_category'].unique():
    median = df[df['locality_category'] == gr]['center_nearest'].median()
    df.loc[df['locality_category'] == gr] = df.loc[df['locality_category'] == gr].fillna(median)
    
# переведем расстояние до центра в целочисленный тип
df['center_nearest'] = df['center_nearest'].astype('int')
# полная проверка пропусков
df.info()

**Выводы**. Выполнено:
1. Заполнение пропусков (столбцы _last_price, is_apartment, balcony, locality_name, floors_total, rooms, kitchen_area, living_area, ceiling_height, days_exposition, airports_nearest, parks_around3000, ponds_around3000, parks_nearest, ponds_nearest_ )
2. Приведение типов (столбцы _last_price, is_apartment, balcony, floors_total, days_exposition, parks_around3000, ponds_around3000,first_day_exposition_ )
3. Удаление выбросов (столбцы _ceiling_height_ )
4. Переименование столбца _"cityCenters_nearest"_
5. Устранение несограсованности данных (_locality_name_ )

## 6. Посчитайте и добавьте в таблицу

### 6.1 Добавление в таблицу цены за квадратный метр

In [ ]:
df['meter_price'] = (df['last_price'] / df['total_area']).astype('int')

df[['last_price', 'total_area', 'meter_price']].head()

### 6.2 Добавление в таблицу: день недели, месяц и год публикации объявления

In [ ]:
df['first_year'] = df['first_day_exposition'].dt.year
df['first_month'] = df['first_day_exposition'].dt.month

# прибавим 1 для более понятного представления дня недели
df['first_weekday'] = df['first_day_exposition'].dt.weekday + 1

# проверка
df[['first_day_exposition', 'first_year', 'first_month', 'first_weekday']].sort_values(by='first_year', ascending=False).head(10)

### 6.3 Добавление в таблицу: этаж квартиры

In [ ]:
df['floors_category'] = df.apply(lambda x: set_floor_category(x.floor, x.floors_total), axis=1)
df['floors_category_num'] = df.apply(lambda x: set_floor_category_num(x.floor, x.floors_total), axis=1)

# проверка
df['floors_category'].value_counts()

In [ ]:
# проверка
df['floors_category_num'].value_counts()

### 6.4 Добавление в таблицу: соотношение жилой и общей площади

In [ ]:
df['living_area_part'] = (df['living_area'] / df['total_area']).round(2)

# проверка
df[['living_area', 'total_area', 'living_area_part']].head()

### 6.5 Добавление в таблицу: соотношение площади кухни к общей

In [ ]:
df['kitchen_area_part'] = (df['kitchen_area'] / df['total_area']).round(2)

# проверка
df[['kitchen_area', 'total_area', 'kitchen_area_part']].head()


### Вывод

Выполнено насыщение данных. Добавлено:
* Цена за квадратный метр
* День недели, месяц и год публикации объявления
* Категория этажа квартиры
* Соотношение общей и жилой площади
* Соотношение общей площади и кухни

## 7. Исследовательский анализ данных

### 7.1 Изучение параметров: площадь, цена, число комнат, высота потолков

**Площадь**

In [ ]:
get_data_info(df, 'total_area', dsc=True, boxplot=True, hist=True)

<font color=green> Автоматизация процесса построения графиков осуществлена круто, молодец. Число bins следует уменьшить. Иначе появляются такие вот лишние пики на графике. 
    
---

In [ ]:
# смотрим на выбросы
df[(df['total_area'] >= 500)]['total_area'].count()

In [ ]:
# Удалим выбросы - 9 объектов с площадью >= 500 м
df = df.drop(df[df['total_area'] >= 500 ].index, axis=0)

# проверка
get_data_info(df, 'total_area', dsc=True, boxplot=True, hist=True)

Медиана отличается от среднего значения на 8 кв. метров. Стандартное отклонение большое - 34. Это объясняется тем, что в данных есть несколько объектов с большой площадью. График демонстрирует выбросы в данных со значением площади >= 500 м, который был удалён. 

**Цена**

In [ ]:
get_data_info(df, 'last_price', dsc=True, boxplot=True, hist=True)

In [ ]:
# количество объектов с ценой > 150 000 000
df[df['last_price'] > 150000000]['last_price'].count()

In [ ]:
df = df.drop(df[df['last_price'] > 150000000].index, axis=0)

get_data_info(df, 'last_price', dsc=True, boxplot=True, hist=True)

Медиана отличается от среднего значения на 2 миллиона. Стандартное отклонение большое - больше медианы. Это объясняется тем, что в данных есть несколько объектов с высокой ценой. Удалены выбросы - объекты дороже 150 000 000 у.е.

**Число комнат**

In [ ]:
get_data_info(df, 'rooms', dsc=True, boxplot=True, hist=True)

In [ ]:
# количество объектов, у которых число комнат > 10
df[df['rooms'] > 10]['rooms'].count()

In [ ]:
# удаляем объекты, у которых число комнат > 10
df = df.drop(df[df['rooms'] > 10.].index, axis=0)

get_data_info(df, 'rooms', dsc=True, boxplot=True, hist=True)

Согласно описанию, медиана и среднее почти равны - 2-комнатные квартиры. Наиболее популярные представители квартир в объявлениях - 1-комнатные и 2-комнатные. В данных несколько объектов с количеством комнат больше 10 - выбросы, удалёны.

**Высота потолков**

In [ ]:
get_data_info(df, 'ceiling_height', dsc=True, boxplot=True, hist=True)

In [ ]:
# количество объектов с высотой потолка 2 < x < 6
df[(df['ceiling_height'] > 6) | (df['ceiling_height'] < 2)]['ceiling_height'].count()

In [ ]:
# удалим выбросы
df = df.drop(df.query('ceiling_height > 6 or ceiling_height < 2').index, axis=0)

# гистограмма после изменений
# df[['ceiling_height']].hist(bins=30)
get_data_info(df, 'ceiling_height', dsc=True, boxplot=True, hist=True)

По графику видно, что в данных о высоте потолка есть выбросы - значения больше 6 метров, которые удалены.

**Выводы**

Удалены выбросы в столбцах:
* total_area
* last_price
* rooms
* ceiling_height


<div class="alert alert-success">
<h2> Комментарий наставника</h2>

Выбросы из данных удалены. Ты выбрала очегь мягкие условия удаления выбросов. Лучше их ужесточить. Все же представить себе квартиру с 15 комнатами довольно сложно. 
</div>

<font color=green>Данный раздел работы заметно улучшен. Ты привела все необходимые графики, а также сделала условия отбора значений более жесткими. Так ты откинем больше выбросов, а результаты анализа будут качественнее. 
    
---

### 7.2 Изучение параметров: время продажи квартиры

Для изучения времени продажи квартиры добавим в таблицу столбец с датой продажи.

In [ ]:
df['last_day_exposition'] = df.apply(
    lambda x: x.first_day_exposition + pd.Timedelta(days=x.days_exposition), axis=1)

# проверка
df[['first_day_exposition', 'last_day_exposition', 'days_exposition']].head()

In [ ]:
df['last_year'] = df['last_day_exposition'].dt.year
df['last_month'] = df['last_day_exposition'].dt.month

# прибавим 1 для более понятного представления дня недели
df['last_weekday'] = df['last_day_exposition'].dt.weekday + 1

# проверка
df[['last_day_exposition', 'last_year', 'last_month', 'last_weekday']].sort_values(by='last_year', ascending=False).head(10)

In [ ]:
# посмотрим на значения, где указан срок объявления
days_exposition = df[df['days_exposition']>0]

# days_exposition[['days_exposition']].describe()
get_data_info(days_exposition, 'days_exposition', dsc=True, boxplot=True, hist=True)

In [ ]:
days_exposition_stat = days_exposition[days_exposition['days_exposition'] < 285]

# days_exposition_stat[['days_exposition']].boxplot()
get_data_info(days_exposition_stat, 'days_exposition', dsc=False, boxplot=True, hist=False)

In [ ]:
# посмотрим на количество аномально малых значений - меньше 20 дней
get_data_info(days_exposition_stat[days_exposition_stat['days_exposition'] < 20], 
              'days_exposition', dsc=False, boxplot=False, hist=True)

***Вывод*

Были рассморены данные о сроке продажи квартиры, за исключением 0. Выявлено, что в данных есть квартиры, которые были проданы быстро, и квартиры, которые не продавались в течение долгого времени:
* быстро - меньше 3 дней
* долго - больше 285 дней

### 7.3 Факторы, влияющие на стоимость квартиры

In [ ]:
first_last_floor = df[df.floors_category.isin(["первый", "последний"])]

# матрица корреляции
price_corr = first_last_floor[['last_price', 'meter_price', 'total_area', 'rooms', 'floors_category_num', 
                                'center_nearest', 'first_weekday', 'first_month', 'first_year']].corr()

price_corr

In [ ]:
# визуализация матрицы корреляции
corr_visual(price_corr)

In [ ]:
# посмотрим на зависимость общей цены от требуемых параметров

keys = ['total_area', 'rooms', 'floors_category_num', 'center_nearest', 
        'first_weekday', 'first_month', 'first_year']
  
get_price_depend(first_last_floor, keys)

In [ ]:
# посмотрим на зависимость цены за метр от требуемых параметров

keys = ['total_area', 'rooms', 'floors_category_num', 'center_nearest', 
        'first_weekday', 'first_month', 'first_year']

get_price_depend(first_last_floor, keys, ylabel='meter_price')

In [ ]:
# "Выберите 10 населённых пунктов с наибольшим числом объявлений. 
# Посчитайте среднюю цену квадратного метра в этих населённых пунктах.

locality_list = df.groupby('locality_name')['locality_name'].count().sort_values(ascending=False)[:10].index
locality_list_stat = df[df.locality_name.isin(locality_list)]

locality_stat_mean = locality_list_stat.groupby('locality_name')['meter_price'].mean().round(2).sort_values(ascending=False)
locality_stat_mean

In [ ]:
# график зависимости цены от местоположения
p = locality_stat_mean.plot(x='locality_name', y='meter_price')
p.set_xticklabels(p.get_xticklabels(), rotation=45)

Выявлена зависимость цены за квартиру от параметров:
* общая площадь (сильная зависимостьб r=0.71)
* число комнат (0.40)
* этаж (первый или последний)(0.10)
* удалённость от центра (-0.23)

Цена не зависит от параметров:
* день недели размещения объявления
* месяц размещения объявления
* год размещения объявления

Корреляция между ценой и этими параметрами представлена на графике.

In [ ]:
price_corr['last_price'].sort_values().plot(kind='bar', grid=True)

Среди выделенного списка 10 населенных пунктов с наибольшим количеством объявлений максимальная средняя цена за квадратный метр в Санкт-Петербурге, минимальная - в Выборге.

### 7.4 Нахождение области центра Санкт-Петербурга

In [ ]:
# Изучите предложения квартир: для каждой квартиры есть информация о расстоянии до центра

center_nearest_stat = df.query('center_nearest != 0')
center_nearest_stat.head()

In [ ]:
for group, data in center_nearest_stat.groupby('locality_category'):
    print('Гистограмма {}-{}'.format(group, 'center_nearest'))
    data[['center_nearest']].plot.hist(bins=30)
    plt.show()
#     center_nearest_stat = df.query('center_nearest != 0')['center_nearest'].hist(bins=50)

In [ ]:
# Выделите квартиры в Санкт-Петербурге ('locality_name').
piter_stat = center_nearest_stat.query('locality_name == "санкт-петербург"')
piter_stat.head()

In [ ]:
# столбец с расстоянием до центра в километрах
piter_stat['center_nearest_km'] = (piter_stat['center_nearest'] / 1000).astype('int')

# среднюю цену для каждого километра
piter_stat['price_km'] = (piter_stat['last_price'] / piter_stat['center_nearest_km']).round(0)
piter_stat['price_km'].head()

In [ ]:
# Постройте график: он должен показывать, как цена зависит от удалённости от центра. 
# Определите границу, где график сильно меняется — это и будет центральная зона. "
piter_stat.sort_values(by='center_nearest').plot(x='center_nearest_km', 
                                                        y='price_km',
                                                        ylim=0.5,
                                                        grid=True, 
                                                        figsize=(12,10),
                                                        kind='line')

Центр в Санкт Петербурге - в радиусе 7 км.

### 7.5 Анализ сегмента квартир в центре города

In [ ]:
# сегмент квартир в центре
piter_center_stat = piter_stat.query('center_nearest_km <= 7')
piter_center_stat.head()

In [ ]:
# анализ территории по параметрам:
# 
# площадь, цена, число комнат, высота потолков, 
# этаж, удалённость от центра, дата размещения объявления

piter_center_details = piter_center_stat[['last_price', 'ceiling_height', 'rooms', 'total_area',
                                          'floor','center_nearest','first_year']]

# матрица корреляции
piter_center_details_corr = piter_center_details.corr()
piter_center_details_corr

In [ ]:
# визуализация матрицы корреляции
corr_visual(piter_center_details_corr)

In [ ]:
keys = ['ceiling_height', 'rooms', 'total_area', 'floor','center_nearest','first_year']

get_price_depend(piter_center_stat, keys)

In [ ]:
# для всего города
# анализ территории по параметрам: площадь, цена, число комнат, высота потолков

piter_details = (df.query('locality_name == "санкт-петербург"')[[
    'last_price', 'meter_price', 'ceiling_height', 'rooms', 'total_area', 'floor','center_nearest','first_year']])

# матрица корреляции
piter_details_corr = piter_details.corr()
piter_details_corr

In [ ]:
# визуализация матрицы корреляции
corr_visual(piter_details_corr)

In [ ]:
# графики зависимости общей цены во всем городе от указанных параметров
get_price_depend(piter_details,
                 ['ceiling_height','rooms','total_area','floor','center_nearest','first_year'])

In [ ]:
# графики зависимости цены за метр во всем городе от указанных параметров
get_price_depend(piter_details,
                 ['ceiling_height','rooms','total_area','floor','center_nearest','first_year'],
                 ylabel='meter_price')

### Вывод

<!-- # Сделайте выводы. Отличаются ли они от общих выводов по всему городу?" -->

Для центра Санкт-Петербурга матрица корреляции демонстрирует, что цена зависит от параметров:
* общая площадь - __r=0.70__
* количество комнат - __r=0.40__
* высота потолков - __r=0.27__
* расстяние до центра - __r=-0.22__ (чем дальше от центра, тем дешевле квартиры)

Цена слабо зависит от этажа и года размещения объявления. Ниже представлен график, где для каждого параметра отражен коэффициент корреляции.

In [ ]:
piter_center_details_corr['last_price'].sort_values().plot(kind='bar', grid=True)

Для всего города Санкт-Петербург матрица корреляции демонстрирует, что цена зависит от параметров:
* общая площадь - __r=0.71__
* количество комнат - __r=0.41__
* высота потолков - __r=0.34__
* расстяние до центра - __r=-0.30__ (чем дальше от центра, тем дешевле квартиры)

Цена слабо зависит от этажа (-0.02) и года размещения объявления (-0.05). Ниже представлен график, где для каждого параметра отражен коэффициент корреляции.

In [ ]:
piter_details_corr['last_price'].sort_values().plot(kind='bar', grid=True)

При сравнении зависимости цена квартиры в центре города и во всем городе было выявлено, что вне центра более выражена зависимость цены от растояния до центра. Также чуть заметнее зависимость от количества комнат и общей площади.

## 8. Общий вывод

Выполнены 4 этапа работы:

1. Ознакомление с данными


2. Предобработка данных. Выполнено заполнение пропусков, приведение типов, удаление выбросов, переименование столбцов, устранение несограсованности данных.


3. Добавление данных:
    * цена за квадратный метр
    * день недели, месяц и год публикации объявления
    * Категория этажа квартиры
    * Соотношение общей и жилой площади
    * Соотношение общей площади и кухни


4. Исследовательский анализ данных

    Выяснено, что для всего города Санкт-Петербург что цена зависит от:
    * общей площади
    * количества комнат
    * высоты потолков
    * расстяния до центра (чем дальше от центра, тем дешевле квартиры)
    
    Цена слабо зависит от этажа и года размещения объявления.

    При сравнении зависимости цена квартиры в центре города и во всем городе было выявлено, что вне центра более выражена зависимость цены от растояния до центра. Также чуть заметнее зависимость от количества комнат и общей площади.